### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42965248,15699,79085,51530,0,0,1149,5,2025-07-17 19:35,2025-07-17 19:35,0,0,2062,5,1
34,42965214,51582,79085,3,0,0,1149,6,2025-07-18 19:33,2025-07-18 19:34,0,0,8186,2,1
35,42969635,54147,79085,51531,0,0,1150,0,2025-07-19 19:35,2025-07-19 19:35,2,0,4965,3,1
36,42740312,23775,79085,1349,9,71,1150,1,2025-07-20 21:58,2025-07-20 21:59,3,0,28504,4,1
37,42969607,31717,79085,51530,0,0,1150,2,2025-07-21 19:30,2025-07-21 19:30,0,2,3633,3,1
38,42969611,137121,79085,51530,0,0,1150,3,2025-07-22 15:30,2025-07-22 15:30,2,1,2117,4,1
39,42740328,89569,79085,1349,10,71,1150,4,2025-07-23 21:59,2025-07-23 21:59,2,1,12067,4,1
40,42970690,20095,79085,51530,0,0,1150,5,2025-07-24 18:30,2025-07-24 18:30,4,0,6341,5,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

8

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,15699,41832,33208,0,4,2,1,0,442,9,9,8,42965248
1,79085,28120,36744,1,11,1,1,0,4-2-3-1a,11,10,11,42965248
0,51582,19112,44017,0,11,0,0,0,nowa,10,9,8,42965214
1,79085,51027,26122,0,0,2,1,0,5-4-1a,5,7,7,42965214
0,54147,29481,35905,2,12,0,0,0,kojaktag2,10,11,12,42969635
1,79085,40469,34045,1,5,3,1,0,4-3-3a,10,10,11,42969635
0,23775,13395,43539,1,17,1,0,0,serie D,11,12,9,42740312
1,79085,56603,26459,5,2,1,1,0,3-4-3SP,8,10,10,42740312
0,31717,34283,33343,0,7,4,1,0,4-3-3-,10,10,9,42969607
1,79085,35746,36686,0,10,0,0,0,4-3-3a,10,10,11,42969607


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1880

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,23775,10,25,8,1,1,27,10,25517027008011,1349
1,122169,10,24,8,0,2,55,13,24542055008008,1349
2,6276,10,23,7,2,1,31,5,23526031007012,1349
3,78751,10,20,6,2,2,18,6,20512018006002,1349
4,79587,10,19,6,1,3,28,10,19518028006001,1349
5,124139,10,17,5,2,3,29,12,17517029005010,1349
6,23755,10,17,5,2,3,23,15,17508023005009,1349
7,78671,10,15,5,0,5,24,20,15504024005007,1349
8,89569,10,6,2,0,8,15,32,6483015002004,1349
9,79866,10,4,1,1,8,18,21,4497018001005,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()